In [ ]:
import torch
import copy
import time
import gc
import torch.cuda.comm

In [ ]:
torch.autograd.set_grad_enabled(False)

In [ ]:
test1 = torch.nn.Linear(25000, 25000, bias=False).half()
test2 = copy.deepcopy(test1)
test3 = torch.nn.Linear(25000, 25000, bias=False).half()
test4 = copy.deepcopy(test3)

In [ ]:
foo = torch.tensor(0).to("cuda")
initial_memory = torch.cuda.memory_allocated()/(1024**3)
test1.to("cuda")
test3.to("cuda")
final_memory = torch.cuda.memory_allocated()/(1024**3)

In [ ]:
print("With pinned memory, detach, broadcast")
for param in test2.parameters():
    param.data = param.data.detach().pin_memory() #With detatch
    #param.data.pin_memory() #No detatch
    gc.collect()
    torch.cuda.empty_cache()
for param in test4.parameters():
    param.data = param.data.detach().pin_memory() #With detatch
    #param.data.pin_memory() #No detatch
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
for param1 in test1.parameters():
    param1.data *= 0 
    param1.data += 1

for param1 in test3.parameters():
    param1.data *= 0 
    param1.data += 1

In [ ]:
num_loops = 30

start_time = time.time()
extrastream = torch.cuda.Stream(device=0,priority = -1)
with torch.no_grad():
    for i in range(num_loops):
        #with torch.cuda.stream(cudastreams[0]):

        for param1,param2 in zip(test1.parameters(),test2.parameters()):
            with torch.cuda.stream(extrastream):
                torch.cuda.comm.broadcast(param2.data,out = [param1.data])

        for param1,param2 in zip(test3.parameters(),test4.parameters()):
            torch.cuda.comm.broadcast(param2.data,out = [param1.data])

        torch.cuda.synchronize()

del extrastream

end_time = time.time()
bandwidth = num_loops * (final_memory - initial_memory) / (end_time - start_time)
print("time for ",num_loops, " is ", (end_time - start_time), " seconds")
print("total memory transferred was ",num_loops * (final_memory - initial_memory), " gb")
print("bandwidth is ",bandwidth , "Gbps")